This is a simple script that ranks the players every year based on their overall rating score. It then returns a table with all those players falling in the Top 20 ranking per year.
Note that there will be years with less or more than 20 players in the rankings. Obviously, the total number of players ranked in the top 20 will vary depending on how many players have the same overall rating.

I'm going to keep experimenting and adding some graphs etc later on when I have some time.

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import datetime

# Reading in the data from the SQLite database file
db = 'soccer.sqlite'
connect = sqlite3.connect(db)
query = "SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(query, connect)

,name
0,sqlite_sequence
1,Player_Attributes
2,Player
3,Match
4,League
5,Country
6,Team
7,Team_Attributes


In [3]:
# Put data into dataframes
query = "SELECT * FROM player;"
players_df = pd.read_sql(query, connect)

query = "SELECT * FROM player_attributes"
player_stats_df = pd.read_sql(query, connect)

# Set options so I can see more of the dataframe
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)

Data manipulation

In [4]:
# Merge the player and player_attribute data
df = players_df.merge(player_stats_df, how='inner', on='player_api_id')

# Calculate the age of the players
df['birthday'] = pd.to_datetime(df['birthday'])
df['date'] = pd.to_datetime(df['date'])
df['age'] = df['date'] - df['birthday']

# Drop unnecessary columns
ratings_df = df.drop(['id_x', 'id_y', 'player_fifa_api_id_x', 'player_fifa_api_id_y',
                     'birthday', 'attacking_work_rate', 'defensive_work_rate'], 1)
# ratings_df = df[['player_api_id', 'player_name', 'age', 'height', 'weight', 'date',
#                  'overall_rating', 'potential']]

# Drop players without any rating
ratings_df = ratings_df.drop(ratings_df[ratings_df['overall_rating'].isnull()].index)

# Sorting by rating rather than age
# I will later group by the year and want to have the highest rating per player when I
# remove the duplicate players within each year
ratings_df.sort_values(['player_name', 'player_api_id', 'overall_rating',
                        'potential'], ascending=[True, True, False, False], inplace=True)

# Change the date to just the year
ratings_df['date'] = ratings_df['date'].apply(lambda x: x.year)

Grouping by year

In [5]:
# Grouping the players by the year
group = ratings_df.groupby('date')

# Dropping the duplicate player entries per year
ratings_df_unique = group.apply(lambda x: x.drop_duplicates(subset = 'player_api_id', keep = 'first'))

# Grouping the df again into another df group object
group = ratings_df_unique.groupby('date')

Ranking players based on overall ratings

In [6]:
# Adding a rating rank var
# Logic here is that if overall rating ties are not taken into account, then the data would
# be biased in an arbitrary manner for regression/prediction later on
# For example, if you have the data ranked by rating and then alphabetically, then those players
# at the end of the alphabet will be left out of the top 20, even if they have the same
# overall rating as those players with names lower in the alphabet
ratings_df_unique['rating_rank'] = group['overall_rating'].rank(ascending=False)

In [10]:
# Return the top 20 overall rating ranks by year
output_df = group.apply(lambda x: x.drop(x[x['rating_rank'] > 20].index).sort_values(
    by=['rating_rank', 'potential'], ascending=[True, False]).reset_index(drop=True))

In [14]:
output_df.to_csv('for_vis.csv')